In [6]:
import os
import pandas as pd
import numpy as np
import math
import ast
from util import *
import plotly.express as px
import plotly.graph_objects as go

In [101]:
merged = load_dataframes("merged_sales_ratings")
merged.head()

,Name,Meta_score,User_score,Description,URL,Developer,Type,Rating,Rank,Platform,Year,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Genre
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,singleplayer,E,95,N64,1998,Nintendo,4.10,1.89,1.45,0.16,7.60,"['Action Adventure', 'Fantasy', 'Action']"
1,The Legend of Zelda: Ocarina of Time,94.0,90.0,The Legend of Zelda: Ocarina of Time 3D takes ...,https://www.metacritic.com/game/3ds/the-legend...,GREZZO,singleplayer,E10+,95,N64,1998,Nintendo,4.10,1.89,1.45,0.16,7.60,"['Compilation', 'Open-World', 'Miscellaneous',..."
2,The Legend of Zelda: Ocarina of Time,99.0,91.0,"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,singleplayer,E,269,3DS,2011,Nintendo,2.03,1.27,0.62,0.30,4.21,"['Action Adventure', 'Fantasy', 'Action']"
3,The Legend of Zelda: Ocarina of Time,94.0,90.0,The Legend of Zelda: Ocarina of Time 3D takes ...,https://www.metacritic.com/game/3ds/the-legend...,GREZZO,singleplayer,E10+,269,3DS,2011,Nintendo,2.03,1.27,0.62,0.30,4.21,"['Compilation', 'Open-World', 'Miscellaneous',..."
4,Super Mario Galaxy,97.0,91.0,[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,singleplayer,E,49,Wii,2007,Nintendo,6.16,3.40,1.20,0.76,11.52,"['Platform', 'Action', 'Platformer', '3D']"


In [102]:
#merged["Platform"] = merged["Platform"].apply(str)
#merged["Platform"] = merged["Platform"].apply(ast.literal_eval)
merged["Genre"] = merged["Genre"].apply(str)
merged["Genre"] = merged["Genre"].apply(ast.literal_eval)
merged = merged[merged["Year"] > 1970]
merged = merged[merged["Year"] < 2016]

In [65]:
print(merged["Year"].min())
print(merged["Year"].max())

1980
2015


In [66]:
max_size = merged["Name"].apply(len).max()
merged["Name"] = merged["Name"].apply(lambda x:x.rjust(50, " "))

In [109]:
fig = px.bar(
    merged.sort_values(by=["Year",'Global_Sales'], ascending = [True,False]).groupby("Year").head(10).sort_values(by=['Global_Sales'], ascending = True),
    x="Global_Sales",
    y="Name",
    animation_frame="Year", 
    range_x=[0, 35],
    barmode="group",
    orientation="h"
)
#for f in fig.frames:
#    f.layout.update(xaxis_range = [0,35])
#fig.update_layout(yaxis_range=[0,10])
#fig.update_layout(xaxis_range=[0,35])
fig.update_layout(margin=dict(l=400))
#fig.update_layout(
#    xaxis = dict(
#        tickmode = 'array',
#        tickvals = [0, 5, 10, 15, 20, 25, 30, 25],
#        ticktext = [0, 5, 10, 15, 20, 25, 30, 25]
#    )
#)
#fig.update_layout(
#    xaxis = dict(
#        range = [0,35],
#        type = 'linear'
#    )
#)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1500
fig.show()